In [1]:
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Datasets

/content/drive/.shortcut-targets-by-id/1Xvmy4JPQNukFURFjRgJQjgR3wNkeCSSK/Datasets


In [3]:
#!git clone https://github.com/Lightning-AI/lit-llama

Cloning into 'lit-llama'...
remote: Enumerating objects: 1982, done.
remote: Counting objects: 100% (57/57), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 1982 (delta 15), reused 14 (delta 2), pack-reused 1925
Receiving objects: 100% (1982/1982), 1.71 MiB | 9.34 MiB/s, done.
Resolving deltas: 100% (1222/1222), done.


In [8]:
!ls

FluencyBank   LibriStutter  n-hypothesis.ipynb	tokenizer.model  wav2vec2.0.ipynb
libristutter  lit-llama     SEP-28k		UCLASS		 whisper.ipynb


In [9]:
!pip install -r lit-llama/requirements.txt

  Cloning https://github.com/Lightning-AI/lightning (to revision master) to /tmp/pip-install-mnyn_9ar/lightning_215cc85fc14a477e97ad67fa1de67bb1
  Running command git clone --filter=blob:none --quiet https://github.com/Lightning-AI/lightning /tmp/pip-install-mnyn_9ar/lightning_215cc85fc14a477e97ad67fa1de67bb1
  Resolved https://github.com/Lightning-AI/lightning to commit 896c2a656ad2db3278ec11520aed04e378f4462b
  Running command git submodule update --init --recursive -q
  Encountered 22 file(s) that should have been pointers, but weren't:
        .notebooks/course_UvA-DL/01-introduction-to-pytorch.ipynb
        .notebooks/course_UvA-DL/02-activation-functions.ipynb
        .notebooks/course_UvA-DL/03-initialization-and-optimization.ipynb
        .notebooks/course_UvA-DL/04-inception-resnet-densenet.ipynb
        .notebooks/course_UvA-DL/05-transformers-and-MH-attention.ipynb
        .notebooks/course_UvA-DL/06-graph-neural-networks.ipynb
        .notebooks/course_UvA-DL/07-deep-energy

In [3]:
import os
from dataclasses import dataclass, field, replace
from typing import TYPE_CHECKING, Dict, Iterable, List, Optional, Sequence, Tuple, Union
import torch
import torch
import torch.nn.functional as F
from torch import Tensor

import pandas as pd
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import torchaudio

processor = WhisperProcessor.from_pretrained("openai/whisper-medium.en")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-medium.en")
model = model.to("cuda")

#model = whisper.load_model("large-v2")
model.eval()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/805 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.41M [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/1.83k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.95k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.06G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/1.95k [00:00<?, ?B/s]

WhisperForConditionalGeneration(
  (model): WhisperModel(
    (encoder): WhisperEncoder(
      (conv1): Conv1d(80, 1024, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(1024, 1024, kernel_size=(3,), stride=(2,), padding=(1,))
      (embed_positions): Embedding(1500, 1024)
      (layers): ModuleList(
        (0-23): 24 x WhisperEncoderLayer(
          (self_attn): WhisperSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=False)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias

In [4]:
# you may replace this with train, test or val specifically

data = pd.read_csv("/content/drive/MyDrive/Datasets/libristutter/outputs/whisper-medium-inference.csv")

In [5]:
data

,ground_truth,reference,path
0,Jean Valjean sat opposite them on chilling on ...,[' and Jean Valjean sat opposite them. Aunt Ji...,audio/5393-19219-0009.flac
1,I'm back all is well now listen can you hear y...,"["" I am back. All is well. Now listen. I am. W...",audio/5678-43303-0036.flac
2,the cottage in the New Forest was held by and ...,"[' The cottage in the new forest was held by, ...",audio/1334-135589-0104.flac
3,I kept the library as long as I could we can s...,[' I kept the library as long as I could. We c...,audio/4018-103416-0010.flac
4,and entertained him hospitably not knowing tha...,"[' and entertained him hospitably, not knowing...",audio/2289-152254-0021.flac
...,...,...,...
387,did the crowd loved to behold that monstrous l...,[' That the crowd loves to behold that monstro...,audio/5393-19218-0039.flac
388,I believe that it will be better for me refrai...,[' I believe that it will be better for me to ...,audio/311-124404-0001.flac
389,chapter 1 mrs. Rachel Lynde is surprised mrs. ...,"["" CHAPTER I. Mrs. Rachel Lynde is Surprised M...",audio/103-1240-0000.flac
390,then he paid a visit to the landlord and hande...,"["" Then he paid a visit to the landlord and ha...",audio/2836-5355-0048.flac


In [6]:
# TODO: check that data is properly readable and fit into test_data var used in sample code

#clean data

import ast

def clean_reference(string):
  return ast.literal_eval(string)

data['inference'] = data['reference'].apply(clean_reference)

test_data = data

In [7]:
test_data

,ground_truth,reference,path,inference
0,Jean Valjean sat opposite them on chilling on ...,[' and Jean Valjean sat opposite them. Aunt Ji...,audio/5393-19219-0009.flac,[ and Jean Valjean sat opposite them. Aunt Jil...
1,I'm back all is well now listen can you hear y...,"["" I am back. All is well. Now listen. I am. W...",audio/5678-43303-0036.flac,[ I am back. All is well. Now listen. I am. We...
2,the cottage in the New Forest was held by and ...,"[' The cottage in the new forest was held by, ...",audio/1334-135589-0104.flac,"[ The cottage in the new forest was held by, a..."
3,I kept the library as long as I could we can s...,[' I kept the library as long as I could. We c...,audio/4018-103416-0010.flac,[ I kept the library as long as I could. We ca...
4,and entertained him hospitably not knowing tha...,"[' and entertained him hospitably, not knowing...",audio/2289-152254-0021.flac,"[ and entertained him hospitably, not knowing ..."
...,...,...,...,...
387,did the crowd loved to behold that monstrous l...,[' That the crowd loves to behold that monstro...,audio/5393-19218-0039.flac,[ That the crowd loves to behold that monstrou...
388,I believe that it will be better for me refrai...,[' I believe that it will be better for me to ...,audio/311-124404-0001.flac,[ I believe that it will be better for me to r...
389,chapter 1 mrs. Rachel Lynde is surprised mrs. ...,"["" CHAPTER I. Mrs. Rachel Lynde is Surprised M...",audio/103-1240-0000.flac,[ CHAPTER I. Mrs. Rachel Lynde is Surprised Mr...
390,then he paid a visit to the landlord and hande...,"["" Then he paid a visit to the landlord and ha...",audio/2836-5355-0048.flac,[ Then he paid a visit to the landlord and han...


In [10]:
import sys
from pathlib import Path
import torch
import requests
import json
import os

# note requires installing lit_llama dependencies from above
from lit_llama.tokenizer import Tokenizer
from tqdm import tqdm

In [11]:
tokenizer_path: Path = Path("tokenizer.model")
tokenizer = Tokenizer(tokenizer_path)
train_set =test_data
print(f"train has {len(train_set):,} samples")
print("Processing train split ...")

train has 392 samples
Processing train split ...


In [18]:
!pip install whisper_openAI

In [21]:
!pip install openai-whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.6/798.6 kB 4.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 22.9 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20231117-py3-none-any.whl size=801358 sha256=509fbcc446423bf8085d7a3f1e2aec2b69b1a5f4e117b4f8bc4b9965a27ee70a
  Stored in directory: /root/.cache/pip/wheels/d0/85/e1/9361b4cbea7dd4b7f6702fa4c3afc94877952eeb2b62f45f56
Successfully built openai-whisper


In [23]:
import whisper

In [32]:
result = []

def tokenize(tokenizer: Tokenizer, string: str, max_length: int, eos=True) -> torch.Tensor:
    return tokenizer.encode(string, bos=True, eos=eos, max_length=max_length)

instruction = 'You are an ASR transcript selector. You have a few transcripts generated by an automatic speech recognition model. Your task is to generate the most likely transcript from them. If the generated transcripts have grammatical or logical errors, you will modify them accordingly to produce the most accurate and coherent transcript.'
for i, ip in test_data.iterrows():
    print(i+1, ' / ', len(test_data))
    inference = ip['inference']
    gt = ip['ground_truth']

    # Removing the ground_truth, if present among the inferences for the prompt
    if gt in inference:
        inference.remove(gt)

        # joining the inputs with '\n'
    for_input = '\n'.join(inference[:10])

    # The prompt follows the Aplaca template ; https://github.com/tatsu-lab/stanford_alpaca
    full_prompt = f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n ### Instruction:\n{instruction}\n\n### Input:\n{for_input}\n\n### Response:"""
    full_prompt_and_response = full_prompt + gt

    encoded_full_prompt = tokenize(tokenizer, full_prompt, max_length=2048, eos=False)
    encoded_full_prompt_and_response = tokenize(tokenizer, full_prompt_and_response, eos=True, max_length=2048)
    labels = encoded_full_prompt_and_response.clone()
    labels_with_masked_input = encoded_full_prompt_and_response.clone()
    labels_with_masked_input[:len(encoded_full_prompt)] = -1


    path = 'libristutter/' + ip['path']
    audio = whisper.load_audio(path)
    audio = whisper.pad_or_trim(audio)
    mel = whisper.log_mel_spectrogram(audio).to(model.device)  # of shape (#channels =  80 ; #samples = )
    mel = mel.unsqueeze(0)

    with torch.no_grad():
        audio_features  = model.model.encoder(mel)

    result.append( {**ip,'index':i, "input_ids": encoded_full_prompt_and_response, "input_ids_no_response": encoded_full_prompt, "labels": labels, 'labels_with_masked_input': labels_with_masked_input,'audio_features': audio_features.last_hidden_state})


0  /  392
1  /  392
2  /  392
3  /  392
4  /  392
5  /  392
6  /  392
7  /  392
8  /  392
9  /  392
10  /  392
11  /  392
12  /  392
13  /  392
14  /  392
15  /  392
16  /  392
17  /  392
18  /  392
19  /  392
20  /  392
21  /  392
22  /  392
23  /  392
24  /  392
25  /  392
26  /  392
27  /  392
28  /  392
29  /  392
30  /  392
31  /  392
32  /  392
33  /  392
34  /  392
35  /  392
36  /  392
37  /  392
38  /  392
39  /  392
40  /  392
41  /  392
42  /  392
43  /  392
44  /  392
45  /  392
46  /  392
47  /  392
48  /  392
49  /  392
50  /  392
51  /  392
52  /  392
53  /  392
54  /  392
55  /  392
56  /  392
57  /  392
58  /  392
59  /  392
60  /  392
61  /  392
62  /  392
63  /  392
64  /  392
65  /  392
66  /  392
67  /  392
68  /  392
69  /  392
70  /  392
71  /  392
72  /  392
73  /  392
74  /  392
75  /  392
76  /  392
77  /  392
78  /  392
79  /  392
80  /  392
81  /  392
82  /  392
83  /  392
84  /  392
85  /  392
86  /  392
87  /  392
88  /  392
89  /  392
90  /  392
91  /  39

In [33]:
# NOTE: REPLACE FILE NAME WITH CORRECT SPLIT BEFORE RUNNING OR ELSE OVERWRITE
torch.save(result,'/content/drive/MyDrive/Datasets/libristutter/outputs/libristutter_test.pt')